In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [49]:

# Load pre-trained VGG19 model
vgg = models.vgg19(pretrained=True).features.eval()


c:\Users\Mahmoud\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Mahmoud\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [51]:
# Define content and style layers
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']


In [52]:

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = torch.tensor(mean).view(-1, 1, 1).to(device)
        self.std = torch.tensor(std).view(-1, 1, 1).to(device)

    def forward(self, img):
        self.mean = self.mean.to(img.device)
        self.std = self.std.to(img.device)
        return (img - self.mean) / self.std

# Mean and standard deviation of ImageNet dataset
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


In [53]:

# Create normalization module
normalization = Normalization(mean, std).to(device)


In [54]:

# Load and preprocess images
def load_image(image_path, transform=None, max_size=None, shape=None):
    image = Image.open(image_path)
    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.ANTIALIAS)
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    if transform:
        image = transform(image)
    image = transforms.ToTensor()(image).unsqueeze(0)
    return image.to(device)

# Content and style loss functions
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input
    
# Gram matrix calculation
def gram_matrix(input):
    batch_size, channels, height, width = input.size()
    features = input.view(batch_size * channels, height * width)
    G = torch.mm(features, features.t())
    return G.div(batch_size * channels * height * width)



In [55]:
content_img= load_image("brown-and-red-concrete-building-3075532.jpg")
style_img=load_image("starry-night-1093721_1280.jpg")

In [56]:
# Model with content and style layers
content_losses = []
style_losses = []

# Move the normalization module to the GPU
normalization.to(device)

model = nn.Sequential(normalization)
model.to(device)  # Move the model to the GPU

i = 0
for layer in vgg.children():
    if isinstance(layer, nn.Conv2d):
        i += 1
        name = 'conv{}_{}'.format(i, i)
    elif isinstance(layer, nn.ReLU):
        name = 'relu{}_{}'.format(i, i)
        layer = nn.ReLU(inplace=False)
    elif isinstance(layer, nn.MaxPool2d):
        name = 'pool_{}'.format(i)
    elif isinstance(layer, nn.BatchNorm2d):
        name = 'bn_{}'.format(i)
    else:
        raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

    layer = layer.to(device)  # Move the layer to the GPU

    model.add_module(name, layer)

    if name in content_layers:
        # Move content_img to GPU
        content_img = content_img.to(device)
        target = model(content_img).detach()
        content_loss = ContentLoss(target)
        model.add_module("content_loss_{}".format(i), content_loss)
        content_losses.append(content_loss)

    if name in style_layers:
        # Move style_img to GPU
        style_img = style_img.to(device)
        target_feature = model(style_img).detach()
        style_loss = StyleLoss(target_feature)
        model.add_module("style_loss_{}".format(i), style_loss)
        style_losses.append(style_loss)


In [57]:
target =load_image("OIP.jpeg")
# Remove the layers after the last content and style losses
for i in range(len(model) - 1, -1, -1):
    if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
        break

model = model[:(i + 1)]

# Optimization parameters
optimizer = torch.optim.Adam([target], lr=0.03, betas=[0.5, 0.999])
num_steps = 500
style_weight = 1000000
content_weight = 1

# Style transfer
for step in range(num_steps):
    target_features = model(target)
    content_loss = 0
    style_loss = 0

    for cl in content_losses:
        content_loss += cl.loss
    for sl in style_losses:
        style_loss += sl.loss

    content_loss *= content_weight
    style_loss *= style_weight

    total_loss = content_loss + style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # if step % 50 == 0:
    #     print("Step [{}/{}], Content Loss: {:.4f}, Style Loss: {:.4f}".format(step + 1, num_steps, content_loss, style_loss.item()))


In [61]:

# Save the stylized image
utils.save_image(target, 'stylized_image.png')

In [62]:
# plt.imshow(target.to(device))